In [4]:
from ultralytics import YOLO
# 훈련된 모델 로드
model_fall = YOLO(r'.\fall.pt')
model_fire = YOLO(r'.\fire.pt')
model_equip = YOLO(r'.\equipment.pt')

# 이미지 예측 (단일 모델)

In [13]:
import cv2
import logging
from ultralytics import YOLO

# YOLO 로그 억제
logging.getLogger('ultralytics').setLevel(logging.ERROR)

def run_multiple_yolo_on_image(image_path, models, threshold=0.3, save_path=None):
    """
    여러 YOLO 모델을 이용해 이미지 1장에 대한 예측을 수행하고 결과를 시각화 + 저장
    Args:
        image_path (str): 입력 이미지 경로
        models (list): YOLO 모델 인스턴스 리스트
        threshold (float): 탐지 신뢰도 임계값
        save_path (str or None): 결과 이미지를 저장할 경로 (None이면 저장 안함)
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"이미지를 불러올 수 없습니다: {image_path}")

    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255),
        (0, 255, 255), (255, 0, 255), (128, 128, 128)
    ]

    for idx, model in enumerate(models):
        results = model(image, verbose=False)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                confidence = box.conf.item()
                class_id = int(box.cls.item())
                label = model.names[class_id]

                if confidence >= threshold:
                    color = colors[idx % len(colors)]
                    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    cv2.putText(image, f'{label} {confidence:.3f}', 
                                (int(x1), int(y1) - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                0.8, color, 2)

    cv2.imshow('YOLO Multi-model Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    if save_path:
        cv2.imwrite(save_path, image)
        print(f"결과 이미지가 저장되었습니다: {save_path}")


In [ ]:
# 모델 미리 로드
model_fall = YOLO(r'.\fall.pt')
model_fire = YOLO(r'.\fire.pt')
model_equip = YOLO(r'.\equipment.pt')

# 모델 리스트
model_list = [model_fall, model_fire, model_equip]

# 이미지 경로
image_path = r'.\test.jpg'
save_path = r'.\result.jpg'  # 결과 저장 경로

# 실행
run_multiple_yolo_on_image(image_path, model_list, threshold=0.5, save_path=save_path)


# 영상 예측하기

In [15]:
import cv2
import logging
from tqdm import tqdm
from ultralytics import YOLO

# YOLO 로그 억제
logging.getLogger('ultralytics').setLevel(logging.ERROR)

def run_multiple_yolo_on_video(video_path, models, threshold=0.3, save_path=None):
    """
    여러 YOLO 모델을 사용해 동영상에 대해 프레임 단위로 예측, 시각화 및 저장
    Args:
        video_path (str): 입력 동영상 경로
        models (list): YOLO 모델 인스턴스 리스트
        threshold (float): 탐지 신뢰도 임계값
        save_path (str or None): 결과 동영상을 저장할 경로 (None이면 저장 안 함)
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"동영상을 열 수 없습니다: {video_path}")

    frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 저장 설정
    if save_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(save_path, fourcc, fps, (frame_width, frame_height))
    else:
        out = None

    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255),
        (0, 255, 255), (255, 0, 255), (128, 128, 128)
    ]

    with tqdm(total=total_frames, desc='Processing frames') as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            for idx, model in enumerate(models):
                results = model(frame, verbose=False)

                for result in results:
                    boxes = result.boxes
                    for box in boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        confidence = box.conf.item()
                        class_id = int(box.cls.item())
                        label = model.names[class_id]

                        if confidence >= threshold:
                            color = colors[idx % len(colors)]
                            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                            cv2.putText(frame, f'{label} {confidence:.3f}',
                                        (int(x1), int(y1) - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX,
                                        0.8, color, 2)

            # 저장 또는 실시간 출력
            if out:
                out.write(frame)

            cv2.imshow('YOLO Multi-model Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            pbar.update(1)

    cap.release()
    if out:
        out.release()
        print(f"결과 동영상이 저장되었습니다: {save_path}")
    cv2.destroyAllWindows()


In [ ]:
model_fall = YOLO(r'.\fall.pt')
model_fire = YOLO(r'.\fire.pt')
model_equip = YOLO(r'.\equipment.pt')

model_list = [model_fall, model_fire, model_equip]

video_path = r'.\falled.mp4'
save_path = r'.\output.mp4'

run_multiple_yolo_on_video(video_path, model_list, threshold=0.5, save_path=save_path)


# 카메라로 실시간 예측

In [17]:
import cv2
import logging
from ultralytics import YOLO

# YOLO 로그 억제
logging.getLogger('ultralytics').setLevel(logging.ERROR)

def run_multiple_yolo_from_instances(models, threshold=0.3):
    """
    이미 생성된 YOLO 모델 인스턴스를 받아 웹캠에서 실시간 객체 탐지를 수행
    Args:
        models (list): YOLO 모델 인스턴스 리스트
        threshold (float): 신뢰도 임계값
    """
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("카메라를 열 수 없습니다.")

    # 모델별 색상
    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255),
        (0, 255, 255), (255, 0, 255), (128, 128, 128)
    ]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("카메라에서 영상을 가져올 수 없습니다.")
            break

        for idx, model in enumerate(models):
            results = model(frame, verbose=False)

            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf.item()
                    class_id = int(box.cls.item())
                    label = model.names[class_id]

                    if confidence >= threshold:
                        color = colors[idx % len(colors)]
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                        cv2.putText(frame, f'{label} {confidence:.3f}',
                                    (int(x1), int(y1) - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    0.9, color, 2)

        cv2.imshow('YOLO Multiple Models (Instances)', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [20]:
# YOLO 모델 로드
model_fall = YOLO(r'.\fall.pt')
model_fire = YOLO(r'.\fire.pt')
model_equip = YOLO(r'.\equipment.pt')

# 모델 리스트
model_list = [model_fire, model_fall, model_equip]

# 실시간 웹캠 예측 실행
run_multiple_yolo_from_instances(model_list, threshold=0.5)
